# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [238]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [239]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [240]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [241]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [242]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [243]:
# Check the result:

A @ x - b

array([ 0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,  0.])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [244]:
def jacob(A, b, rtol=1e-16, niter = 500):
    dgA = np.diag(np.diag(A))
    B = A - dgA
    x = np.ones(b.shape[0])
    dgA_ = np.diag(1/np.diag(dgA))
    n = 0
    for i in range(niter):
        normB = np.linalg.norm(dgA_ @ -B)
        if normB > 1: #checking convergence
            print("don't converge")
            break
        x_s = (dgA_ @ -B ) @ x + dgA_ @ b
        n +=1
        if np.linalg.norm(x_s - x) <= rtol: #elimination of excess iterations
            break
        else:
            x = x_s
    return x, n


In [245]:
n = 10
np.set_printoptions(suppress=True)
for l in range(1,20): #we check some dominance of matrix
    A = rndm.uniform(size=(n, n)) + np.diagflat([l]*n)
    b = rndm.uniform(size=n)
    xx = np.linalg.solve(A,b)
    x, m = jacob(A,b)
    print(np.abs(xx-x),'Diag multiply: ',l, 'Niter: ', m)

don't converge
[1.69028049 0.34674145 1.25212443 0.94810969 0.63289142 1.68956675
 0.76193901 0.92340856 1.04634973 0.04209147] Diag multiply:  1 Niter:  0
don't converge
[0.84394861 0.88037364 0.66600454 0.80867916 0.7869155  1.14538078
 0.73429736 1.22672463 1.12557723 0.99134399] Diag multiply:  2 Niter:  0
don't converge
[0.88492617 1.11121635 0.78623234 1.16486714 0.86765002 1.08494476
 1.04304059 0.83135432 0.76590397 0.91315443] Diag multiply:  3 Niter:  0
don't converge
[1.03429143 1.00009777 0.94169352 0.96024901 1.03918878 1.02263367
 0.92457748 1.01385928 0.83446026 0.85449813] Diag multiply:  4 Niter:  0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Diag multiply:  5 Niter:  169
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Diag multiply:  6 Niter:  113
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Diag multiply:  7 Niter:  71
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Diag multiply:  8 Niter:  66
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Diag multiply:  9 Niter:  52
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Diag multiply:  10 Niter:  40
[0. 0. 

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [246]:
def seidel(A, b, rtol = 1e-16, niter= 500):
    U = np.triu(A,k=1)
    L = np.tril(A,k=0)
    L_ = np.linalg.inv(L)
    x = np.ones(b.shape[0])
    n = 0
    for i in range(niter):
        normB = np.linalg.norm(L_ @ U)
        if normB > 1: #checking convergence
            print("don't converge")
            break
        x_s = L_ @ b - L_@ U @ x
        n += 1 
        if np.linalg.norm(x_s - x)<= rtol:
            break
        x = x_s
    return x,n

In [247]:
#convergence is only guaranteed if the matrix is either diagonally dominant, or symmetric and positive definite
for i in range(10):
    n = 10
    A = rndm.uniform(size=(n, n)) + np.diagflat([5]*n)#diagonally dominant
    b = rndm.uniform(size=n)
    xx = np.linalg.solve(A,b)
    x, k = seidel(A,b)
    print('DDominant: ',np.abs(xx-x),'Niter: ', k)

    a = np.random.rand(n, n)
    M = np.tril(a,-1) + np.tril(a, -1).T + np.diagflat([5]*n)#symmetric and positive definite
    xx1 = np.linalg.solve(M,b)
    x1,p = seidel(M,b)
    print('PSymmetric: ',np.abs(xx1-x1), 'Niter: ', p)

    C = rndm.uniform(size=(n, n)) #random won't converge
    xx2 = np.linalg.solve(C,b)
    x2,u = seidel(C,b)

    print('Random: ', np.abs(xx2-x2), 'Niter: ', u)


DDominant:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  28
PSymmetric:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  23
don't converge
Random:  [0.81075754 2.52057467 2.0665217  0.78188964 1.81430634 0.4546695
 1.42871118 0.04475773 2.83897545 0.5963693 ] Niter:  0
DDominant:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  22
PSymmetric:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  26
don't converge
Random:  [0.98465121 2.2042164  0.09611676 0.59245988 1.57833379 1.28030354
 1.76481199 1.4340043  2.9885939  1.00000378] Niter:  0
DDominant:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  25
PSymmetric:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  29
don't converge
Random:  [5.67939014 3.90833017 1.09557964 4.28201803 0.40121489 5.42195606
 1.23847474 0.00939071 0.08293836 0.39229512] Niter:  0
DDominant:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  22
PSymmetric:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  24
don't converge
Random:  [  1.5090538   44.75717273  31.6173864  120.67020884  33.32795666
  57.29384097   6.96836204

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [248]:
def min_res(A, b, x0, rtol=1e-16, niter = 5000):
    x = x0
    n = 0
    r_ = np.array([])
    for i in range(niter):
        r = A @ x - b
        r_ = np.append(r_, np.linalg.norm(r))
        tau = np.dot(r, A @ r)/np.linalg.norm(A @ r)**2 #this I took from lecture
        x_s = x - tau * r
        n += 1
        if np.linalg.norm(x_s - x)<= rtol:
            break
        x = x_s
    return x, n, r_[0], r_[n-1]

In [250]:
for i in range(10):
    n = 10
    A = rndm.uniform(size=(n, n)) + np.diagflat([5]*n)#diagonally dominant
    b = rndm.uniform(size=n)
    x0 = np.ones(b.shape[0])
    xx = np.linalg.solve(A,b)
    x,k, r11, r12 = min_res(A, b, x0)
    print('DDominant: ',np.abs(xx-x), 'Niter: ', k, 'Starts / Ends:',r11/r12)

    a = np.random.rand(n, n)
    M = np.tril(a,-1) + np.tril(a, -1).T + np.diagflat([2]*n)#symmetric and positive definite
    xx1 = np.linalg.solve(M,b)
    x1, p, r21, r22 = min_res(M, b, x0)
    print('pSymmetric: ',np.abs(xx1-x1), 'Niter: ', p, 'Starts / Ends:',r21/r22)

    C = rndm.uniform(size=(n, n)) #random won't converge
    xx2 = np.linalg.solve(C,b)
    x2, u, r31, r32 = min_res(C,b, x0)
    print('Random: ', np.abs(xx2-x2), ', Niter: ', u)

DDominant:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  41 Starts / Ends: 8.168676459837272e+16
pSymmetric:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  425 Starts / Ends: 5.217662107505157e+16
Random:  [1.56615036 1.72278205 2.77799364 0.37563815 0.57704058 1.98565004
 1.64372807 2.51556843 0.55966319 1.42618178] , Niter:  15
DDominant:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  34 Starts / Ends: 1.2044812980960003e+17
pSymmetric:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  1013 Starts / Ends: 3.877452483174474e+16
Random:  [0.07567072 1.24545175 2.22189477 1.75292102 0.71172562 1.11264196
 0.87253602 1.82292117 2.77911588 0.49331605] , Niter:  9
DDominant:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  37 Starts / Ends: 3.9809057076666936e+16
pSymmetric:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Niter:  214 Starts / Ends: 7.880490563620981e+16
Random:  [0.61681528 0.32587598 0.60642315 1.65580175 1.19628238 0.22615603
 1.01381782 0.983003   1.89244565 0.13150679] , Niter:  36
DDominant:  [0. 0. 0. 0. 0. 0. 0. 0